<a href="https://colab.research.google.com/github/kimberlyydanij/jimat/blob/main/%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/seoulfood

Mounted at /content/drive
/content/drive/MyDrive/seoulfood


In [ ]:
!pip install selenium
!pip install -U  urllib3 requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/

In [ ]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']

In [ ]:
# 25개구 for문으로 돌려서 카페 정보 크롤링하기

import os
from time import sleep
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

##########################################################################
##################### variable related selenium ##########################
##########################################################################
# 서울 특별시 구 리스트
gu_list = ['강남구']


# csv 파일에 헤더 만들어 주기

for index, gu_name in enumerate(gu_list):
    k=0
    fileName = '강남구url.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8-sig')
    file.write("url" + "|" +"구" + "|" + "ID" + "|" +"맛집명" + "|" + "타입" +"|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "|" + "평점" +"\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')        # Head-less 설정
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=options)
    # options.add_argument('headless')
    options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36   ")
    options.add_argument('lang=ko_KR')
    chromedriver_path = "D:/lsy/chromedriver.exe"
    #driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
    driver.get('https://map.kakao.com/')  # 주소 가져오기
     #1.search텍스트창 가져오기
    search_area = driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 맛집')  # 검색어 입력
    #2.검색누르기
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    #3.더보기란 클릭하기
    more_page = driver.find_element(By.ID,"info.search.place.more")
    # more_page.click()
    #more_page.send_keys(Keys.ENTER) # 더보기 누르고
    driver.execute_script("arguments[0].click();",more_page)
    # print(more_page)
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    next_btn = driver.find_element(By.ID,"info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
      
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)

        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR,"#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH,xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(1)
            place_lists = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.place\.list > li')
            for p in place_lists: 
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # print(place_name)

                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_type = store_info.select('.head_item > .subcategory')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_address2 = place_address.replace(',', '')
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_hour2 = place_hour.replace(',', '')
                place_tel = store_info.select('.info_item > .contact > span')[0].text
                place_star = store_info.select('div.rating.clickArea > span.score > em')[0].text
                url = store_info.select("div.info_item > div.contact.clickArea > a.moreview")
                for j in url:
                  link = j["href"]
                  print(link)
                  file.write(link+"|")
                
  
                # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR,'div.info_item > div.contact.clickArea > a.moreview')
                
                detail.send_keys(Keys.ENTER)

                #상세탭으로 전환
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(1)
                place_photo = ""
                try:
                    photo = driver.find_element(By.CSS_SELECTOR,'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(1)
                print(place_name, place_photo,place_star)
                k+=1
                file.write("MA"+ "|"+str(k)+"|"+place_name + "|" + place_type +"|" + place_address2 + "|" + place_hour2 + "|" + place_tel + "|" + place_photo + "|"+ place_star +"\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID,"info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            driver.execute_script("arguments[0].click();",next_btn)
    print("End of Crawl")


WebDriverException: ignored